# Running ndmg : One Subject

This tutorial provides a basic overview of how to run ndmg manually within Python.
The absolute easiest way is to run the pipeline from the command line once all dependencies are installed using the following command:

`ndmg_bids </absolute/input/dir> </absolute/output/dir>`.

This will run a single session from the input directory, and output the results into your output directory.

### Steps
1. First, we grab some test data and atlases from our github repository. <br>
2. Then, we choose our input parameters (the defaults work fine if you don't want to worry about this!) <br>
3. Last, we run the pipeline.

Running the pipeline is quite simple: call `ndmg_dwi_pipeline.ndmg_dwi_worker` with the correct input flags.

Let's begin!

In [21]:
import os
import os.path as op
import glob
import shutil
import warnings
import subprocess
from pathlib import Path

from ndmg.scripts import ndmg_dwi_pipeline
from ndmg.utils import s3_utils

## Download test data, atlases, and check for dependencies.

The below code will grab some sample diffusion MRI data from our `neuroparc` repository, as well as the atlases ndmg needs to run. <br>
If you want to explore these data and atlases, you can find it in `~/.ndmg/`.

Note that the below code requires `git lfs` to be installed.

#### Make sure that AFNI and FSL are installed

In [38]:
# FSL
try:
    print(f"Your fsl directory is located here: {os.environ['FSLDIR']}")
except KeyError:
    raise AssertionError("You do not have FSL installed! See installation instructions here: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FslInstallation")
    
# AFNI
try:
    print(f"Your AFNI directory is located here: {subprocess.check_output('which afni', shell=True, universal_newlines=True)}")
except subprocess.CalledProcessError:
    raise AssertionError("You do not have AFNI installed! See installation instructions here: https://afni.nimh.nih.gov/pub/dist/doc/htmldoc/background_install/main_toc.html")

Your fsl directory is located here: /usr/local/fsl
Your AFNI directory is located here: /Users/alex/abin/afni



#### Test Data

In [ ]:
# Make data directory if it doesn't exist
ndmg_dir = Path.home() / ".ndmg/"
data_dir = ndmg_dir / "data/"

# Remove old data if it exists
if data_dir.is_dir():
    shutil.rmtree(data_dir)
    
# Remove neuroparc if it already exists
if Path("neuroparc").is_dir():
    shutil.rmtree("neuroparc")
    
# Clone the sample data into `~/.ndmg`.
data_dir.mkdir(parents=True)
!git lfs clone https://github.com/neurodata/neuroparc.git
shutil.move("neuroparc/data/BNU1", data_dir)
data_dir = data_dir / "BNU1"

#### Atlases

In [ ]:
# Remove old atlas dir if it exists
atlas_dir = ndmg_dir / "ndmg_atlases"
if atlas_dir.is_dir():
    shutil.rmtree(atlas_dir)

# Download atlases to atlas_dir
atlas_dir.mkdir(parents=True)
for name in Path("neuroparc").iterdir():
    shutil.move(str(name), atlas_dir)

In [ ]:
# Remove our now-empty directory
shutil.rmtree("neuroparc")

## Choose input parameters

#### Naming Conventions
Here, we define input variables to the pipeline.
To run the `ndmg` pipeline, you need four files:
1. a `t1w` - this is a high-resolution anatomical image.
2. a `dwi` - the diffusion image.
3. bvecs - this is a text file that defines the gradient vectors created by a DWI scan.
4. bvals - this is a text file that defines magnitudes for the gradient vectors created by a DWI scan.

The naming convention is in the [BIDs](https://bids.neuroimaging.io/) spec.

In [ ]:
# Specify base directory and paths to input files (dwi, bvecs, bvals, and t1w required)
subject_id = 'sub-0025864'

# Define the location of our input files.
t1w = str(data_dir / f"{subject_id}/ses-1/anat/{subject_id}_ses-1_T1w.nii.gz")
dwi = str(data_dir / f"{subject_id}/ses-1/dwi/{subject_id}_ses-1_dwi.nii.gz")
bvecs = str(data_dir / f"{subject_id}/ses-1/dwi/{subject_id}_ses-1_dwi.bvec")
bvals = str(data_dir / f"{subject_id}/ses-1/dwi/{subject_id}_ses-1_dwi.bval")

print(t1w)
print(dwi)
print(bvecs)
print(bvals)

#### Parameter Choices
Here, we choose the parameters to run the pipeline with.
If you are inexperienced with diffusion MRI theory, feel free to just use the default parameters.

- *atlases = ['desikan', 'CPAC200', 'DKT', 'HarvardOxfordcort', 'HarvardOxfordsub', 'JHU', 'Schaefer2018-200', 'Talairach', 'aal', 'brodmann', 'glasser', 'yeo-7-liberal', 'yeo-17-liberal']* : The atlas that defines the node location of the graph you create.
- *mod_types = ['det', 'prob']* : Deterministic or probablistic tractography.
- *track_types = ['local', 'particle']* : Local or particle tracking.
- *mods = ['csa', 'csd']* : [Constant Solid Angle](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4360965/) or [Constrained Spherical Deconvolution](https://onlinelibrary.wiley.com/doi/10.1002/ima.22005).
- regs = *['native', 'native_dsn', 'mni']* : Registration style. If native, do all registration in each scan's space; if mni, register scans to the MNI atlas; if native_dsn, do registration in native space, and then fit the streamlines to MNI space.
- vox_size = *['1mm', '2mm']* : Whether our voxels are 1mm or 2mm.
- seeds = int : Seeding density for tractography. More seeds generally results in a better graph, but at a much higher computational cost.

In [ ]:
# Use the default parameters.
atlas = 'desikan'
mod_type = 'det'
track_type = 'local'
mod_func = 'csd'
reg_style = 'native'
vox_size = '2mm'
seeds = 20

# Set an output directory
outdir = '/tmp/output_{}_{}_{}_{}_{}_{}_{}'.format(atlas, mod_type, track_type, mod_func, seeds, reg_style, subject_id)
print(f"Your output directory will be : {outdir}")

#### Get masks and labels
The pipeline needs these two variables as input. <br>
Running the pipeline via `ndmg_bids` does this for you.

In [ ]:
# Auto-set paths to neuroparc files
mask = str(atlas_dir / "atlases/mask/MNI152NLin6_res-2x2x2_T1w_descr-brainmask.nii.gz")
labels = [str(i) for i in (atlas_dir / "atlases/label/Human/").glob(f"*{atlas}*2x2x2.nii.gz")]

print(f"mask location : {mask}")
print(f"atlas location : {labels}")

## Run the pipeline!

In [ ]:
ndmg_dwi_pipeline.ndmg_dwi_worker(dwi=dwi, bvals=bvals, bvecs=bvecs, t1w=t1w, atlas=atlas, mask=mask, labels=labels, outdir=outdir, vox_size=vox_size, mod_type=mod_type, track_type=track_type, mod_func=mod_func, seeds=seeds, reg_style=reg_style, clean=False, skipeddy=True, skipreg=True)

## Run in parallel

In [ ]:
'''
Import nipype utilities (must install nipype)
'''
from nipype.pipeline import engine as pe
from nipype.interfaces import utility as niu
from nipype.interfaces.base import BaseInterface, BaseInterfaceInputSpec, TraitedSpec, File, traits, SimpleInterface

In [ ]:
'''
Construct multi-subject workflow
'''

wf = pe.Workflow(name="NDMG_group_wf")

# Create an import list of modules/objects to permeate the workflow environment
import_list = ["import shutil", "import time", "import os", "import numpy as np", "import networkx as nx",
               "import nibabel as nib", "import warnings", "warnings.filterwarnings(\"ignore\")",
               "np.warnings.filterwarnings(\"ignore\")", "warnings.simplefilter(\"ignore\")", 
               "from subprocess import Popen", "from dipy.tracking.streamline import Streamlines",
              "import ndmg", "from ndmg import preproc as mgp", "from ndmg.utils import gen_utils as mgu", 
               "from ndmg.utils import s3_utils", "from ndmg.register import gen_reg as mgr", 
               "from ndmg.track import gen_track as mgt", "from ndmg.graph import gen_graph as mgg",
              "from ndmg.utils.bids_utils import name_resource", "from ndmg.stats.qa_tensor import *", 
               "from ndmg.stats.qa_fibers import *", "from datetime import datetime"]

inputnode = pe.Node(niu.IdentityInterface(fields=['atlas', 'mask', 'labels', 
                                                  'outdir', 'vox_size', 'mod_type', 
                                                  'track_type', 'mod_func', 'reg_style', 'clean', 
                                                  'skedy', 'skipreg']),
                    name='inputnode', imports=import_list)

inputnode.inputs.atlas = atlas
inputnode.inputs.mask = mask
inputnode.inputs.labels = labels[0]
inputnode.inputs.outdir = outdir
inputnode.inputs.vox_size = vox_size
inputnode.inputs.mod_type = mod_type
inputnode.inputs.track_type = track_type
inputnode.inputs.mod_func = mod_func
inputnode.inputs.seeds = seeds
inputnode.inputs.reg_style = reg_style
inputnode.inputs.clean = False
inputnode.inputs.skedy = False
inputnode.inputs.skipreg = False

# Make NDMG an interface
class ndmgDWIworkerInputSpec(BaseInterfaceInputSpec):
    """
    Input interface wrapper for ndmgDWIworker
    """
    dwi = traits.Str(mandatory=True)
    bvals = traits.Str(mandatory=True)
    bvecs = traits.Str(mandatory=True)
    t1w = traits.Str(mandatory=True)
    atlas = traits.Str(mandatory=True)
    mask = traits.Str(mandatory=True)
    labels = traits.Str(mandatory=True)
    outdir = traits.Str(mandatory=True)
    vox_size = traits.Str(mandatory=True)
    mod_type = traits.Str(mandatory=True)
    track_type = traits.Str(mandatory=True)
    mod_func = traits.Str(mandatory=True)
    seeds = traits.Str(mandatory=True)
    reg_style = traits.Str(mandatory=True)
    clean = traits.Bool(mandatory=False)
    skedy = traits.Bool(mandatory=False)
    skipreg = traits.Bool(mandatory=False)

class ndmgDWIworker(BaseInterface):
    """
    Interface wrapper for ndmgDWIworker
    """
    input_spec = ndmgDWIworkerInputSpec

    def _run_interface(self, runtime):
        out = ndmg_dwi_pipeline.ndmg_dwi_worker(
            self.inputs.dwi,
            self.inputs.bvals,
            self.inputs.bvecs,
            self.inputs.t1w,
            self.inputs.atlas,
            self.inputs.mask,
            self.inputs.labels,
            self.inputs.outdir,
            self.inputs.vox_size,
            self.inputs.mod_type,
            self.inputs.track_type,
            self.inputs.mod_func,
            self.inputs.seeds,
            self.inputs.reg_style,
            self.inputs.clean,
            self.inputs.skedy,
            self.inputs.skipreg)
        setattr(self, '_outpath', out)
        return runtime


# Make the NDMG wf itself a node of the DAG
ndmg_dwi_worker_node = pe.Node(interface=ndmgDWIworker(),
                                  inputs=['dwi', 'bvals', 'bvecs', 't1w',
                                          'atlas', 'mask', 'labels', 
                                          'outdir', 'vox_size', 'mod_type', 
                                          'track_type', 'mod_func', 'seeds', 
                                          'reg_style', 'clean', 'skedy', 'skipreg'],
                                   nested=True, synchronize=True, name='ndmg_dwi_worker_node', imports=import_list)

# Restrict cpu and memory for that node such that scheduling will wait for available 
# resources before proceeding with additional subjects in parallel
ndmg_dwi_worker_node.interface.n_procs = 2
ndmg_dwi_worker_node.interface.mem_gb = 8

# Make subject files iterable
ndmg_dwi_worker_node.iterables = [("dwi", [i[4] for i in input_files]), ("bvals", [i[2] for i in input_files]), ("bvecs", [i[3] for i in input_files]), ("t1w", [i[1] for i in input_files])]

# Connect variables of workflow
wf.connect([
    (inputnode, ndmg_dwi_worker_node, [('atlas', 'atlas'), 
                                       ('mask', 'mask'), 
                                       ('labels', 'labels'), 
                                       ('outdir', 'outdir'), 
                                       ('vox_size', 'vox_size'), 
                                       ('mod_type', 'mod_type'), 
                                       ('track_type', 'track_type'), 
                                       ('mod_func', 'mod_func'), 
                                       ('seeds', 'seeds'), 
                                       ('reg_style', 'reg_style'), 
                                       ('clean', 'clean'), 
                                       ('skedy', 'skedy'), 
                                       ('skipreg', 'skipreg')])
])

# Configure wf execution
cfg = dict(execution={'stop_on_first_crash': True, 'hash_method': 'content', 'crashfile_format': 'txt',
                      'display_variable': ':0', 'job_finished_timeout': 65, 'matplotlib_backend': 'Agg',
                      'use_relative_paths': True, 'parameterize_dirs': False,
                      'remove_unnecessary_outputs': False, 'remove_node_directories': False,
                      'raise_insufficient': True})
for key in cfg.keys():
    for setting, value in cfg[key].items():
        wf.config[key][setting] = value

# Set runtime directory to home directory
wf.base_dir = op.expanduser("~")

# Run wf using multiproc scheduler
wf.run(plugin='MultiProc', plugin_args = {'n_procs': int(2), 'memory_gb': int(16)})